# Mid-Module Assigment - Data PreProcessing

#### Loading the TSV
For now, we don't load the TSV with specific column data types.

From the cursory inspection, it looks like we would need to spend some effort in cleaning the last 2nd last column (column before Criteria). We would come back to it later after sanitising rest of the columns.

In [111]:
import pandas as pd
import numpy as np
import re
import json

df = pd.read_csv('./SIS_faculty_list.tsv', sep='\t', header=0)
# Let's fix the typo "divison" in the column name
df.rename(columns = {'Divison':'Division'}, inplace = True)
df.head(1)

,ID,Name,Location,Grade,Title,Join Date,LWD,Type,Division,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,DOCUMENT OTHER PROFESSIONAL CERTIFICATION CRITIERA Five Years Work Experience Teaching Excellence Professional Certifications,Criteria
0,LT9210743,Jim Hunter,London,FA,Faculty - Business,29-Aug-10,NaN,Full Time,Business,Abbas Houda,Masters,Master of Arts,International Affairs,American University of Beirut,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship BUS200...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience


In [112]:
# Utility functions

def check_is_empty(x):
    if x is None or pd.isna(x) or pd.isnull(x):
        return True
    else:
        return False

def check_is_not_empty(x):
    return not check_is_empty(x)

#### Inspecting for missing values

From the output below, we can see that the series for LWD (Last Working Day) is 96% empty. This is a column we should `drop` - since we can't impute this column for missing values and the missing percentage is **too high**.

In [113]:
missing_percentages = (df.isnull().sum() / len(df)) * 100
for k, v in missing_percentages.items():
    if v > 10:
        print('Column: ' + k + ' has missing value %: ' + str(v))

Column: LWD has missing value %: 96.12676056338029
Column: MAJOR TEACHING FIELD has missing value %: 20.774647887323944
Column: DOCUMENT OTHER PROFESSIONAL CERTIFICATION CRITIERA Five Years Work Experience Teaching Excellence Professional Certifications has missing value %: 21.830985915492956


#### Dropping unneccessary columns - 'LWD'
From the above inspection (for missing values), LWD is chosen to be dropped.

In [114]:
df.drop('LWD', axis=1, inplace=True)
print('LWD column is dropped')

LWD column is dropped


#### Dropping unneccessary columns - 'ID' and 'Name'
I want to drop the `ID` and `Name` columns as well, since its a unique identifier and not useful for ML purposes. But before dropping the `ID` column, I want to make sure that there are `no duplicate rows` for `ID` and `Name` - which could `potentially help in imputing missing values`

As we can see below, `no 2 IDs are repeated`. 

In [115]:
# Filter all rows where ID = NaN or '0'
temp_check = df.dropna()
temp_check = temp_check.loc[temp_check['ID'] != '0']
ids = temp_check['ID']
temp_check[ids.isin(ids[ids.duplicated()])]

,ID,Name,Location,Grade,Title,Join Date,Type,Division,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,DOCUMENT OTHER PROFESSIONAL CERTIFICATION CRITIERA Five Years Work Experience Teaching Excellence Professional Certifications,Criteria


As an additional check, we can also look at name duplications - which has only two instances - both of which, are for distinct persons (by looking at the data in rest of the columns)

In [116]:
# Lets see if any two rows have same name
names = df['Name']
df[names.isin(names[names.duplicated()])]

,ID,Name,Location,Grade,Title,Join Date,Type,Division,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,DOCUMENT OTHER PROFESSIONAL CERTIFICATION CRITIERA Five Years Work Experience Teaching Excellence Professional Certifications,Criteria
44,LT9276009,Amanda Hobson,Manchester,FA,Faculty - Business,18-Aug-13,Full Time,Business,Ian Varley,Doctorate,Ph.D,Business Management,Univ of the Sunshine Coast,"Ph.D (Business Management), Post Graduate Cert...",BUS2203 - Business Stats for Managers BUS2303 ...,Economics/ Business Administration,"PGCE, 11 years teaching experience",Industry & Teaching experience
45,LT9004065,Amanda Hobson,NaN,FA,Faculty - Business,16-Sep-02,Full Time,Business,Ian Varley,Doctorate,Doctorate in Marketing,Marketing,University of Westminster,Master of Business Administrat (Business Admin...,CSD1023 - The Crime of Smuggling BUS4563 - Str...,Business Administration/ Marketing,ICDL IRB Instructional Skills Workshops (ISW) ...,Industry & Teaching experience
131,LT9234672,Zikida Koudou,Cardiff,FA,Faculty - Business,26-Aug-12,Full Time,Business,Harriet Mensah,Masters,Masters,Hospitality Management,University of Houston,"Bachelor (Hotel Restaurant Tourism Admin), Mas...",BUS1403 - Business Ethics and Corporate BUS120...,Economics/Management/Marketing/Tourism/Hospita...,Yrs of Teaching Experience - 15 years Prof...,Certification
144,LT9328996,Zikida Koudou,Belfast,FA,Faculty - Business,9-Aug-15,Full Time,Business,Orlamei Princess,Doctorate,Ph.D,Economics Science,University of Montenegro,"Ph.D (Economics Science), Bachelor (Ecological...",BUS3103 - Internat Busi Globalisation BSD2252 ...,NaN,NaN,Industry & Teaching experience


In [117]:
df.drop('ID', axis=1, inplace=True)
print('ID column is dropped')

df.drop('Name', axis=1, inplace=True)
print('Name colum is dropped')

ID column is dropped
Name colum is dropped


#### Inspecting For Typos - Location ... Major
To do this, we simply take a frequency count, and manually inspect occurences which have low values (less than 5) - since typos are often not repeated.

In [118]:
column_names = ['Location', 'Grade', 'Title', 'Type', 'Division', 
                'Highest Qualification Level', 'Highest Qualification', 'Major']
for column_name in column_names:
    freqs = df[column_name].value_counts()
    for k, count in freqs.items():
        if count < 5:
            print('Low frequency count: ' + str(count) + ' for the value: ' + k + ' in column: ' + column_name)
    print('----')

Low frequency count: 4 for the value: Glasgow in column: Location
Low frequency count: 3 for the value: Birmingham in column: Location
Low frequency count: 3 for the value: Southampton in column: Location
Low frequency count: 2 for the value: Cardiff  in column: Location
Low frequency count: 2 for the value: Brighton in column: Location
----
Low frequency count: 2 for the value: Chair in column: Grade
----
Low frequency count: 2 for the value: Faculty- Business in column: Title
Low frequency count: 1 for the value: Faculty - Business (Admin) in column: Title
Low frequency count: 1 for the value: Graduate Trainee in column: Title
Low frequency count: 1 for the value: Faculty - Business  in column: Title
Low frequency count: 1 for the value: Faculty - Business & Comp Lit. in column: Title
Low frequency count: 1 for the value: Faculty - Business & Comp Lit Faculty in column: Title
Low frequency count: 1 for the value: Business, Comp & Math Faculty in column: Title
Low frequency count: 1 f

From the above, we can note the following things - 

*Columns which are OK*
1. Location
2. Grade
3. Type
4. Division

-----

*Columns which are NOT OK*
1. `Title` series has too many variations in writing the same thing - **we need to clean this column** (more on this in further sections)
2. `Highest Qualification Level` also requires multiple cleanups - for e.g. `Ph. D. ...`, `PhD.`, `Doctorate` etc., `Master of`, `Masters in` etc., mean the same things - **we need to clean this column** (more on this in further sections)

-----

*Columns which cannot be inspected at the moment (too many results)*

`Highest Qualification`, `Major` `All Qualifications from Profile` have too many results to check individually - we need to apply other techniques for more efficient cleanups

#### Cleaning up 'Title' column

Lets revisit the frequency count in title again

In [119]:
df['Title'].value_counts()

Faculty - Business                       252
Faculty - Trainee                         13
Adjunct Faculty - Business                 9
Faculty- Business                          2
Faculty - Business (Admin)                 1
Graduate Trainee                           1
Faculty - Business                         1
Faculty - Business & Comp Lit.             1
Faculty - Business & Comp Lit Faculty      1
Business, Comp & Math Faculty              1
Faculty-Business                           1
Faculty-Business (HR & Admin.)             1
Name: Title, dtype: int64

In [120]:
# Cleaning up a few things
df['Title'] = df['Title'].str.replace('Faculty- B', 'Faculty - B')
df['Title'] = df['Title'].str.replace('Faculty-B', 'Faculty - B')
df['Title'] = df['Title'].str.replace('Admin.', 'Admin', regex=False)
df['Title'] = df['Title'].str.replace('Admin', 'Administration', regex=False)
df['Title'] = df['Title'].str.replace('Comp Lit.', 'Comparative Literature', regex=False)
df['Title'] = df['Title'].str.replace('Comp Lit', 'Comparative Literature')
df['Title'] = df['Title'].str.replace('Comp & Math', 'Computing and Math')
df['Title'] = df['Title'].str.strip()
df['Title'].value_counts()

Faculty - Business                                     256
Faculty - Trainee                                       13
Adjunct Faculty - Business                               9
Business, Computing and Math Faculty                     1
Faculty - Business (Administration)                      1
Graduate Trainee                                         1
Faculty - Business & Comparative Literature Faculty      1
Faculty - Business & Comparative Literature              1
Faculty - Business (HR & Administration)                 1
Name: Title, dtype: int64

From the above output, we still need to cleanup the titles - as the word **Faculty** can appear in the beginnning or at the end. Let us **remove the word Faculty** from the title, and store it in a separate column called **Title Prefix**

In [121]:
def filter_title(row):
    if check_is_empty(row['Title']):
        return None
    if 'adjunct faculty' in row['Title'].lower():
        return 'Adjunct Faculty'
    elif 'faculty' in row['Title'].lower():
        return 'Faculty'
    elif 'graduate trainee' in row['Title'].lower():
        return 'Graduate Trainee'
    else:
        return None
df['Title Prefix'] = df.apply(lambda row : filter_title(row), axis=1)

# Verify if any row is left out
temp_check = df[['Title', 'Title Prefix']]
temp_check[temp_check['Title Prefix'].isna()]

,Title,Title Prefix


Its good that there is no row for which **Title Prefix** was not filled, now let's replace occurrence of **Faculty / Adjunct Faculty** or **Graduate Trainee** in title column with empty string

In [122]:
def title_cleanup(row):
    # Case insenstive replace with empty string
    row['Title'] = re.sub('adjunct faculty', '', row['Title'], flags=re.IGNORECASE)
    row['Title'] = re.sub('faculty', '', row['Title'], flags=re.IGNORECASE)
    row['Title'] = re.sub('graduate trainee', '', row['Title'], flags=re.IGNORECASE)
    # Replace all instances of '-' with empty string
    row['Title'] = row['Title'].replace('-', '')
    row['Title'] = row['Title'].strip()

df.apply(lambda row : title_cleanup(row), axis=1)
df['Title'].value_counts()

Business                             265
Trainee                               13
Business & Comparative Literature      2
                                       1
Business (Administration)              1
Business (HR & Administration)         1
Business, Computing and Math           1
Name: Title, dtype: int64

**Optionally** - We could have further split the title values by **','** or **'&'** since:
1. `[Business, Computing and Math]` and `[Computing and Math, Business]` should be treated as same thing
- To fix that, we could sort (ascending) the order in which terms appear
2. If `[Business (HR & Administration)]` could be further written like this - `[Business (Administration), Business (HR)]` so that ML analysis could derive partial similarity between this and `[Business (Administration)]`

However, in the interest of time (and too low count to meaningfully affect current output) - we can **skip this step**.


#### Imputing 'Highest Qualification Level (HQL)' and 'Highest Qualification (HQ)'

Let us analyse the **HQL - HQ** values below.

In [123]:
df.rename(columns = {'Highest Qualification Level':'HQL'}, inplace = True)
df.rename(columns = {'Highest Qualification':'HQ'}, inplace = True)
df.rename(columns = {'All Qualifications from Profile':'AQP'}, inplace = True)

temp_check = df[['HQL', 'HQ', 'AQP']]
temp_check[temp_check['HQL'].isna()]

,HQL,HQ,AQP
19,NaN,NaN,"Post Graduate Diploma (Political Science), Mas..."
99,NaN,NaN,"Bachelor (Hospitality Management), Masters (Bu..."
101,NaN,NaN,"Masters (Finance), Bachelor (Economics)"
245,NaN,NaN,"Bachelor (Accounting), Masters (Business)"
258,NaN,NaN,"Masters (Finance), Bachelor (Accounting & Fina..."
259,NaN,NaN,"Bachelor (English Language & Literature), Mast..."


No scope of imputation for HQL **from HQ**. We can, however, **impute certain values from AQP**. We will **revisit this imputation later**, when we analyze AQP in more detail.

Below is the inverse of above - all rows where HQ is NA.

In [124]:
temp_check[temp_check['HQ'].isna()]

,HQL,HQ,AQP
19,NaN,NaN,"Post Graduate Diploma (Political Science), Mas..."
99,NaN,NaN,"Bachelor (Hospitality Management), Masters (Bu..."
101,NaN,NaN,"Masters (Finance), Bachelor (Economics)"
245,NaN,NaN,"Bachelor (Accounting), Masters (Business)"
258,NaN,NaN,"Masters (Finance), Bachelor (Accounting & Fina..."
259,NaN,NaN,"Bachelor (English Language & Literature), Mast..."


Same as before. As mentioned earlier, imputation of HQL and HQ **from AQP will be considered later** (after this section).

As a last check, let's see if `len(HQL) > len(HQ)` for any row. This would imply that the data was mistakenly filled (swapped).

In [125]:
temp_check = df[(df['HQL'].str.len() > df['HQ'].str.len()) & (df['HQL'].str.lower() != 'doctorate')]
temp_check[['HQL', 'HQ']]

,HQL,HQ
72,MBA in Banking & Finance,MBA
75,Doctor of Jurisprudence Degree,Doctorate
154,Ph.D in Organization and Management,Doctorate
243,PhD Corporate Governance and Accounting,Doctorate
263,Ph. D. (Business Administration),Doctorate
274,Doctorate of Business Administration,Doctorate
277,Ph.D (Economics),Doctorate


Clearly, we can see that the values of HQL and HQ **have been swapped in multiple places**.

In [126]:
for idx in list(temp_check.index.values):
    swap_temp = df.loc[idx, 'HQ']
    df.at[idx, 'HQ'] = df.loc[idx, 'HQL']
    df.at[idx, 'HQL'] = swap_temp

df[['HQL', 'HQ']].iloc[list(temp_check.index.values)]

,HQL,HQ
72,MBA,MBA in Banking & Finance
75,Doctorate,Doctor of Jurisprudence Degree
154,Doctorate,Ph.D in Organization and Management
243,Doctorate,PhD Corporate Governance and Accounting
263,Doctorate,Ph. D. (Business Administration)
274,Doctorate,Doctorate of Business Administration
277,Doctorate,Ph.D (Economics)


#### Cleaning up 'HQL'

Now, that the imputation is done (wherever possible), let's see if we can clean the column values. From the below data, we can do the following:

1. Any case-insensitive occurrence of `Ph.D`, `PhD`, `Ph. D.`, `PhD.`, `Doctor of`, `Doctorate of` can be labelled as Doctorate.
2. **After Step 1** - Any case-insensitive occurence of `Masters`, `Master`, `MBA` can be labelled as Masters. Note: Its important to note that for the purposes of this ML system, **MBA is treated equivalent to Masters.**
3. **After Step 2** - Any case-insensitive occurence of `Bachelor`, `Bachelors` can be labelled as Bachelor.

In [127]:
df['HQL'].value_counts()

Masters                                                                     157
Doctorate                                                                    58
Ph.D                                                                         43
Bachelor                                                                     12
MBA                                                                           2
Bachelor (Applied Science) in Business administration( Human Resources )      1
Master of International Business                                              1
Maters of Philosophy (Economics)                                              1
Master of Business Administration (MBA) and MA (Economics)                    1
Masters (Business Administration), Master of Philosophy (Business)            1
Master's Degree in Public Relations and Press Office Management               1
Name: HQL, dtype: int64

In [128]:
# Occurrences of the values on the left, will be replaced with the key on the right
replace_map = {
    'T_D' : ['phd.', 'ph.d.', 'ph.d', 'ph. d.', 'ph. d', 'phd', 'doctorate', 'doctor'],
    'T_M' : ['masters', "master's", 'master of', 'master', 'mba', 
                   # this is to fix a typo in the TSV
                   'maters'],
    'T_B' : ['bachelors', 'bachelor']
}
final_value_map = {
    'T_D' : 'Doctorate',
    'T_M' : 'Masters',
    'T_B' : 'Bachelor'
}
def hql_cleanup(row):
    # replacement in order
    for key in ['T_D', 'T_M', 'T_B']:
        values = replace_map[key]
        for value in values:
            if check_is_not_empty(row['HQL']) and value in row['HQL'].lower():
                row['HQL'] = key
    
    if check_is_not_empty(row['HQL']):
        row['HQL'] = final_value_map[row['HQL']]

df.apply(lambda row : hql_cleanup(row), axis=1)
df['HQL'].value_counts()

Masters      164
Doctorate    101
Bachelor      13
Name: HQL, dtype: int64

#### Cleaning up 'Highest Qualification'

From the below output, we can observe some general cleanups
1. `Ph.D`, `PhD.` etc could be merged to a single representation (similar to what was done for HQ)
2. `Master`, `Masters`, `Mast` etc in the qualification names should be merged to a single representation
3. `Administrat` should be renamed to `Administration` (and fixing typos like `administrationion`)
4. All names could be `lowercased` to remove differences
5. Replace cells which are just Ph.D or Masters with `na`. This is because, the information is already encapsulated in `HQL` columm. This provides no additional value. 
6. However, we will preserve single letter values like `MBA` (Masters in Business Administration) and `MA` (Master of Arts) as they provide extra info.
7. `of`, `Of` or `in`, `In` cause differences at string level. It would be good if we can think of better representation - which overlooks differences in adverbs/adjectives. For example, a **comma separated tuple representation - (Qualification, Course)** could be more meaningful. In code, we tupulize it to `(Qualification$$Course)` as it is easy to split later on

In [129]:
df['HQ'] = df['HQ'].str.strip()
df['HQ'].value_counts()

Master of Business Administrat                                              73
Ph.D                                                                        63
Master of Science                                                           18
Masters                                                                     17
Master of Commerce                                                           9
                                                                            ..
Ph.D In Accounting                                                           1
MBA                                                                          1
Bachelor (Applied Science) in Business administration( Human Resources )     1
Master of International Business                                             1
MA in Teaching                                                               1
Name: HQ, Length: 66, dtype: int64

In [130]:
# Occurrences of the values on the left, will be replaced with the key on the right
replace_map = {
    'T_P' : ['phd.', 'ph.d.', 'ph.d', 'ph. d.', 'ph. d', 'phd', 'doctorate', 'doctor of'],
    'T_M' : ['masters', "master's", 'master', 'mast', 'mba'],
    'T_B' : ['bachelors', 'bachelor'],
    'T_MB' : ['mba']
}

def basic_replacements(row, col_key):
        for key, values in replace_map.items():
            for value in values:
                if check_is_not_empty(row[col_key]):
                    row[col_key] = row[col_key].lower().replace(value, key.lower())

        # Other cleanup(s)
        if check_is_not_empty(row[col_key]):
            row[col_key] = row[col_key].replace('administrat', 'administration')
            row[col_key] = row[col_key].replace('administrationion', 'administration')
            row[col_key] = row[col_key].replace('admin.', 'administration')
            
def hq_tupulizer(col_key):
    df.apply(lambda row : basic_replacements(row, col_key), axis=1)

    # Replace cells containing ONLY 1 word - t_p, t_m, etc.
    df.replace(r"\bt_p$\b", np.nan, regex=True, inplace=True)
    df.replace(r"\bt_m$\b", np.nan, regex=True, inplace=True)
    df.replace(r"\bt_b$\b", np.nan, regex=True, inplace=True)
    # Replace cells containing ONLY 1 word and 'in' or 'of' - t_p, t_m, etc.
    df.replace(r"\bt_p\s+(of|in)$\b", np.nan, regex=True, inplace=True)
    df.replace(r"\bt_m\s+(of|in)$\b", np.nan, regex=True, inplace=True)
    df.replace(r"\bt_b\s+(of|in)$\b", np.nan, regex=True, inplace=True)
    
    # Note: we are leaving single word cells with `ma`, `t_mb` as it is ^

    # Convert data into a tuple-like represenation (Level$$Qualification)
    df[col_key].replace(
        {r'\b(t_p|t_mb|t_b|t_m|ma)\s+(of\s|in\s)?\(?([\w|\s|&|/]*)\)?' : r'(\1$$\3)'}, 
        regex=True, 
        inplace=True
    )
    
    # Replace empty spaces after '(' or before ')'
    df[col_key].replace(
        {r'(?<=[([]) +| +(?=[)\]])' : ''}, 
        regex=True, 
        inplace=True
    )

hq_tupulizer('HQ')
df['HQ'].value_counts()

(t_m$$business administration)                                        73
(t_m$$science)                                                        18
(t_p$$business administration)                                        10
(t_b$$applied science)                                                 9
(t_m$$commerce)                                                        9
(t_m$$arts)                                                            7
(t_p$$economics)                                                       4
(t_p$$philosophy)                                                      4
(t_m$$philosophy)                                                      4
(t_m$$management)                                                      3
(ma$$international business)                                           3
(t_p$$accounting)                                                      2
(t_m$$law)                                                             2
(t_m$$business)                                    

##### Few notes from above output - 

1. We didn't gain a very high advantage in consolidating outputs, but **atleast** we can easily see the outliers now - which have incorrect pattern
2. For example - `(t_m$$business administration )(t_m) and (ma$$economics)` has two qualifications - which is wrong
3. Some entries like `international business &market`, `management science` don't specify the level.
4. We **can write a more complex logic** for preserving the second half in entries like - 
```
Bachelor (Applied Science) in Business administration( Human Resources )   --> (t_b$$applied science) in business administration(human resources)
```
However, to keep it in common with other entries like `(t_b$$applied science)` - we don't process it further


**Also, note that** - `(t_m$$arts)` and `ma`  ... or `(t_m$$business administration)` and `mba` can be considered equivalent and replaced. (This equivalence however, is not implemented for now)

---

Finally, lets clean and replace the values not matching with our pattern `(Level$$Qualification)` with `na`



In [131]:
hq_search_regex = '\((t_p|t_mb|t_b|t_m|ma)\$\$[\w|\s|&|/]+\)'
def hq_cleaner(row, col_key):
    if check_is_not_empty(row[col_key]):
        temp_str = row[col_key]
        search = re.search(hq_search_regex, temp_str)
        result = search.group(0) if search else ''  # by default, we will only consider the first matching group 
        # - as this column should contain a single qualification
        row[col_key] = result.strip()

df.apply(lambda row : hq_cleaner(row, 'HQ'), axis=1)
# Replace cells with empty string with NA
df.replace(r'^\s*$', np.NaN, regex=True, inplace=True)
df['HQ'].value_counts()

(t_m$$business administration)                                   75
(t_m$$science)                                                   18
(t_p$$business administration)                                   10
(t_b$$applied science)                                           10
(t_m$$commerce)                                                   9
(t_m$$arts)                                                       7
(t_m$$philosophy)                                                 4
(t_p$$philosophy)                                                 4
(t_p$$economics)                                                  4
(t_m$$management)                                                 3
(ma$$international business)                                      3
(t_m$$business)                                                   2
(t_p$$accounting)                                                 2
(t_m$$law)                                                        2
(t_m$$education)                                

#### Data Wrangling AQL

A similar approach (like HQ) needs to followed to **structurize the qualifications**, in the form of a set like:

```
{(t_b$$business administration), (t_b$$applied science), ..}
```

This set could be even **used for data imputation of HQL and HQ!**. We can simply sort the set by qualification prefixes t_b, t_m/t_mb/ma, t_p in order (lowest to highest) and find the highest level as well as the subject.

However, in the interest of time, I am skipping the code implementation for the same.

#### Data Wrangling the "DOCUMENT OTHER ... " column

Let's take a sneak peak from this column

In [132]:
df.rename(columns={
        'DOCUMENT OTHER PROFESSIONAL CERTIFICATION CRITIERA Five Years Work Experience '
        'Teaching Excellence Professional Certifications': 'OtherDoc'}, inplace=True
)

print(df['OtherDoc'].head(10))

0    10+ Years Small Business Ownership, 10+ Years ...
1    years of teaching experience: 17              ...
2    Years of teaching experience: 15 Years of prof...
3    6 years professional experience + 6 years teac...
4    1 year professional experience + 24 years teac...
5                                                  NaN
6                                                  NaN
7                         11 years teaching experience
8    7+ years Teaching/Corporate Training experienc...
9    Taught 1 business course (1 semester)  Have be...
Name: OtherDoc, dtype: object


Its clear that there is no specific structure - and a simple regex won't be sufficient. We could however, try to make a rough **phrase-tokeniser by multiple regex's or some carefully selected separators**. By **phrase-tokens**, I mean small meaningful sentences like `Years of teaching experience: 17`. 

In [133]:
separators = [
    {'name': 'comma', 'regex': False, 'separator': ','},
    {'name': 'semi_colon', 'regex': False, 'separator': ';'},
    {'name': 'dot', 'regex': False, 'separator': '.'},
    {'name': 'tab', 'regex': False, 'separator': '\t'},
    {'name': 'plus', 'regex': False, 'separator': '+'},
    {'name': 'newline', 'regex': False, 'separator': '\n'},
    {'name': 'slash', 'regex': False, 'separator': '/'},
    {'name': 'and', 'regex': False, 'separator': 'and'},
    {'name': 'ampersand', 'regex': False, 'separator': '&'},
    # all regexes below split by capture groups
    {'name': 'regex_1', 'regex': True, 'separator': r'(?i)(.*?experience:\s?\d+)'},
    {'name': 'regex_1', 'regex': True, 'separator': r'(?i)(\d+\s+(years|yrs|year|yr)\s+prof.*?experience)'},
    {'name': 'regex_1', 'regex': True, 'separator': r'(?i)(\d+\s+(years|yrs|year|yr)\s+teaching\s(and\s.*?researcher)?)'},
    {'name': 'regex_2', 'regex': True, 'separator': r'(?i)(\d+.+?(years|yrs|year|yr).+?experience)'},
    {'name': 'regex_2', 'regex': True, 'separator': r'(?i)(.*work experience)\s?and(.*teaching)'},
    {'name': 'regex_3', 'regex': True, 'separator': r'(?i).*?(part time.*)'},
    {'name': 'regex_4', 'regex': True, 'separator': r'(?i)(^(?!,).*(certificate|certification).*(,|;))'},
    {'name': 'regex_5', 'regex': True, 'separator': r'(?i)(,|;)(\s+.*(certificate|certification).*)'},
    {'name': 'regex_whitespace_2+', 'regex': True, 'separator': r'(?:\s{2,})'},
    {'name': 'regex_whitespace_3+', 'regex': True, 'separator': r'(?:\s{3,})'}
]

def phrase_tokenizer(row):
    row_value = row['OtherDoc']
    phrase_tokenize_result = []
    
    if check_is_not_empty(row_value):
        separator_wise_splits = {}
        for separator in separators:
            if not separator['regex']:
                separator_wise_splits[separator['name']] = {
                    'split_values': row_value.split(separator['separator'])
                }
            else:
                separator_wise_splits[separator['name']] = {
                    'split_values': re.split(separator['separator'], row_value)
                }
        
        return {'separator_wise_splits': separator_wise_splits}
        

# Lets see the results
temp_check = df[['OtherDoc']].copy(deep=True)
temp_check['PhraseTokens'] = temp_check.apply(lambda row : phrase_tokenizer(row), axis=1)
for idx, rw in temp_check[temp_check['OtherDoc'].str.contains('experience', na = False)].sample(1).iterrows():
    print(rw['OtherDoc'])
    print(json.dumps(rw['PhraseTokens'], indent=2, sort_keys=True))
    print('------')


years of teaching experience: 8 years of professional/management experience: 12 Professional Certification: Certified Attorney, Certified Notary Public and Certified Conveyancer          
{
  "separator_wise_splits": {
    "ampersand": {
      "split_values": [
        "years of teaching experience: 8 years of professional/management experience: 12 Professional Certification: Certified Attorney, Certified Notary Public and Certified Conveyancer          "
      ]
    },
    "and": {
      "split_values": [
        "years of teaching experience: 8 years of professional/management experience: 12 Professional Certification: Certified Attorney, Certified Notary Public ",
        " Certified Conveyancer          "
      ]
    },
    "comma": {
      "split_values": [
        "years of teaching experience: 8 years of professional/management experience: 12 Professional Certification: Certified Attorney",
        " Certified Notary Public and Certified Conveyancer          "
      ]
    },
   

In the above output, we can observe that some trial and error with regexes ... we are able to catch sub-phrases by some of the **separator** styles. (Note: Since sampling is random in the above statement, results will differ everytime the code is run)

Now, the problem is to weed out the values which are not split, or split uncleanly.

For this we can write a sanitizer, using some guesses and approximations.

1. Remove all results below a specific length (Initially we start with 10 .. we can adjust this through trial and error)
2. Remove all results above a specific lenght (Initially we start with 100 .. we can adjust this through trial and error)
3. Replace `.`, `+`, 'plus`, ':`, `;` with empty string (we don't need stopwords inside phrase-tokens
4. Convert presence of "word-numbers" to numeric representation. For example - "Nine years of teaching experience" would be converted to "9 years of teaching experience"
5. [After step 3] Strip out all the numbers from the phrase-tokens and put them in a separate list. We intend to cluster common results together. Having numerical values would cause unneccessary variance.
6. Remove all instances of multiple whitespaces like `  ` with a single whitespace.
7. Deduplicate splits (phrase-tokens) if multiple separators have given the same output

In [134]:
from utils.text_to_num import text2int

def phrase_token_sanitizer(row):
    row_value = row['PhraseTokens']
    deduplicated_phrases = set()
    if check_is_not_empty(row_value):
        for separator_type, separated_phrases in row_value['separator_wise_splits'].items():
            cleaned_phrases = []
            cleaned_phrases_numerics = []
            for phrase in separated_phrases['split_values']:
                if not phrase:
                    continue
                if len(phrase) <= 10:
                    # 10 is the length of the string '100+ Years'
                    # Too small token to be meaningful
                    continue
                if len(phrase) >= 100:
                    # To err on the safer side, we assume the splitting was unclean (adjust by trial and error)
                    continue

                # Before mutating the selected phrase, we should ideally STORE this - to perform later analysis
                # on which part of the original string WAS NOT selected ..
                # .. (by subtracting selected phrase from the original string)
                phrase = phrase.replace('.', '')
                phrase = phrase.replace(',', '')
                phrase = phrase.replace('+', '')
                phrase = phrase.replace('plus', '')
                phrase = phrase.replace(':', '')
                phrase = phrase.replace(';', '')
                # Replace multiple whitespaces with a single whitespace
                phrase = ' '.join(phrase.split())
                phrase = phrase.strip()

                if phrase:
                    # Convert 'word-numbers' to actual decimal numbers
                    phrase = text2int(phrase)

                    # Replace all numbers by the placeholder 'n_u_m'
                    numerics = re.findall(r'(\d+)', phrase)
                    phrase = re.sub(r'(\d+)', 'n_u_m', phrase)

                    cleaned_phrases_numerics.append(numerics)
                    cleaned_phrases.append(phrase)
                    deduplicated_phrases.add(phrase)

            separated_phrases['split_values'] = cleaned_phrases
            separated_phrases['phrases_numerics'] = cleaned_phrases_numerics

    return deduplicated_phrases


temp_check['CleanedPhraseSet'] = temp_check.apply(lambda row: phrase_token_sanitizer(row), axis=1)
# Lets see the results
for idx, rw in temp_check[temp_check['OtherDoc'].str.contains('experience', na = False)].sample(4).iterrows():
    print('Original String: ' + rw['OtherDoc'] + '\n')
    print('Extracted Phrase Tokens: ' + str(rw['CleanedPhraseSet']))
    print('------')

Original String: 15 years of  professional work in education and business areas;  5  years of  teaching experience

Extracted Phrase Tokens: {'n_u_m years of teaching experience ', 'n_u_m years of professional work in education and business areas ', 'teaching experience ', 'professional work in education and business areas ', 'n_u_m years of professional work in education ', 'n_u_m years of professional work in education and business areas n_u_m years of teaching experience ', 'n_u_m years of ', 'business areas n_u_m years of teaching experience '}
------
Original String: years teaching experience: 10+ management experiece-5 years

Extracted Phrase Tokens: {'years teaching experience n_u_m', 'years teaching experience n_u_m management experiece n_u_m years ', 'management experiece n_u_m years '}
------
Original String: EFQM ISO 9001 Internal Auditor, EFQM ISO 9001 External Auditor, 24 years teaching experience

Extracted Phrase Tokens: {'EFQM ISO n_u_m Internal Auditor EFQM ISO n_u_m E

#### k-Means clustering

(Note: Since sampling is random in the above statement, results will differ everytime the code is run)

The above output looks much better now.

On to the next step: lets run k-means clustering and see if we are able to find some cleanly separated clusters

To run k-means, we need to first vectorize the phrases. k-Means clusterises by distance (euclidean distance) - and hence the notion of a vector - every data point represented as a vector, allows for euclidean calculation between data points.

For vectorising, we will use the "bag of words" approach. We will count the occurences of a particular word in a phrase, and use that for vector representation.

In [135]:
from sklearn.feature_extraction.text import CountVectorizer

# Build a super list
cleaned_phrase_sets = temp_check['CleanedPhraseSet'].tolist()
cleaned_phrases_all = list()
for set_item in cleaned_phrase_sets:
    for item in set_item:
        cleaned_phrases_all.append(item)

# Pass the list to the word vectorizer
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(cleaned_phrases_all).todense()
print(vectorizer.vocabulary_)

{'n_u_m': 273, 'years': 434, 'small': 362, 'business': 63, 'ownership': 296, 'vocational': 421, 'training': 401, 'teaching': 384, 'experience': 150, 'corporate': 101, 'sme': 363, 'consulting': 96, 'of': 289, 'professional': 321, 'management': 253, 'certification': 75, 'gcma': 170, 'acma': 13, 'and': 27, 'active': 14, 'researcher': 338, 'accreditation': 11, 'background': 45, 'year': 433, 'curriculum': 110, 'development': 121, 'certified': 78, 'trainer': 400, 'taught': 382, 'course': 103, 'semester': 349, 'have': 183, 'been': 53, 'doing': 123, 'shadowing': 356, 'observation': 286, 'for': 161, 'now': 284, 'in': 209, 'senior': 351, 'roles': 343, 'higher': 186, 'education': 131, 'months': 270, 'work': 426, 'isw': 232, 'train': 398, 'the': 392, 'fdw': 157, 'icdl': 198, 'excellence': 143, 'award': 44, 'twice': 404, 'institutions': 222, 'more': 271, 'than': 391, 'chartered': 84, 'secretary': 346, 'industry': 214, 'universities': 411, 'colleges': 91, 'socio': 365, 'economic': 127, 'research': 3

In [136]:
# Executing k-means
from sklearn.cluster import KMeans

_cluster_size = 8

km = KMeans(
    n_clusters=_cluster_size, init='random',
    n_init=12, max_iter=10000,
    tol=1e-04, random_state=0
)
labels = km.fit_predict(features)

# To evaluate output, lets create a data-frame with "phrase" and "cluster" together
cluster_map = pd.DataFrame(
    {
        "sentences": cleaned_phrases_all,
        "cluster": km.labels_
    }
)

# Display the output
print("Cluster\t\tSentence")
for index, row in cluster_map.iterrows():
    print(str(row['cluster']) + '\t\t' + str(row['sentences']))

Cluster		Sentence
7		n_u_m Years Small Business Ownership 
7		Vocational Business Training n_u_m Years Teaching Experience 
7		Years Small Business Ownership n_u_m
7		n_u_m Years Corporate/Vocational Business Training 
3		n_u_m Years Small Business Ownership n_u_m Years SME Consulting n_u_m Years Corporate 
7		Years Corporate/Vocational Business Training n_u_m Years Teaching Experience 
7		Years SME Consulting n_u_m
7		n_u_m Years SME Consulting 
7		n_u_m Years Teaching 
7		n_u_m Years Teaching Experience 
5		years of professional/management experience n_u_m Professional Certification GCMA ACMA 
4		years of teaching experience n_u_m years of professional 
6		Certification 
7		years of teaching experience n_u_m
6		ACMA 
5		management experience n_u_m Professional Certification GCMA ACMA 
5		management experience n_u_m
4		Years of teaching experience n_u_m Years of professional 
4		Years of teaching experience n_u_m Years of professional/management experience n_u_m
3		n_u_m years profess

3		n_u_m years professional n_u_m years Teaching Certificate n_u_mV Workplace Training 
7		n_u_m years Teaching Certificate n_u_mV Workplace Training and Assessment 
6		Certificate n_u_mV Workplace Training and Assessment 
5		n_u_m years professional 
6		Assessment 
6		Certificate 
3		n_u_m years professional n_u_m years Teaching Certificate n_u_mV Workplace Training and Assessment 
3		n_u_m years professional n_u_m years Teaching 
7		n_u_m years Teaching 
3		n_u_m years teaching & n_u_m years teaching experience 
6		experience 
7		n_u_m years teaching 
7		n_u_m years teaching experience 
3		n_u_m years professional n_u_m years teaching 
5		n_u_m years professional 
7		n_u_m years teaching 
7		n_u_m years Teaching 
7		n_u_m years Professionaln_u_m Years Teaching 
5		n_u_m years Professional 
7		n_u_m Years Teaching 
5		n_u_m years experience 
5		n_u_m years professional 
3		n_u_m years professional n_u_m years Teaching 
7		n_u_m years Teaching 
5		n_u_m years professional 
3		n_u_m yea

From the above output, it looks like "n_u_m Years Teaching" and similar phrases were rightly put into the same cluster.

We can do more optimisations around regexes, sanitisation, to have cleaner clusters. This is an iterative process.